In [160]:
import torch
import matplotlib.pyplot as plt
import torch.nn.functional as F
%matplotlib inline

In [161]:
words = open('names.txt').read().splitlines()
words[:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In [163]:
vocab = sorted(list(set(''.join(words))))
print(len(vocab))

stoi = {s:i+1 for i,s in enumerate(vocab)}
stoi['.']  = 0

itos = {i:s for s,i in stoi.items()}

print(stoi)
print(itos)

26
{'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, '.': 0}
{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [232]:
X = []
Y = []

blocks_size  = 3
for w in words:
    
    context= [0]*blocks_size
    
    for char in w + '.': 
        ix = stoi[char]
        X.append(context)
        Y.append(ix)
        # print(''.join([ itos[char] for char in context]), '---> ', itos[ix])
        # print(context)
        context = context[1:] + [ix]
        # print(context)
X = torch.tensor(X)
Y = torch.tensor(Y)

In [233]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([228146, 3]), torch.int64, torch.Size([228146]), torch.int64)

In [234]:
C = torch.randn(27,2)

In [235]:
C.shape

torch.Size([27, 2])

In [236]:
F.one_hot(torch.tensor(5), num_classes=27).float().shape

torch.Size([27])

In [237]:
emb = C[X]
print(emb.shape)
# emb
# incase of 3 block size
# emb1 = torch.cat((emb[:, 0 ,:], emb[:, 1, :], emb[:, 2, :]), 1)
# emb1.shape

# emb1 = torch.cat(torch.unbind(emb, 1),1)
# emb1.shape
# emb1.storage()
emb = emb.view(-1,6) 
emb.shape

torch.Size([228146, 3, 2])


torch.Size([228146, 6])

In [238]:
w1 = torch.randn(6, 100)
b1 = torch.randn(100)


In [239]:
a = torch.tanh(emb @ w1 + b1)
a.shape

torch.Size([228146, 100])

In [240]:
w2 = torch.randn(100,27)
b2 = torch.randn(27)

In [241]:
logits = a @ w2 + b2
counts = logits.exp()
prob = counts / counts.sum(1, keepdims=True)
prob.shape
prob

tensor([[1.8319e-06, 1.0000e-10, 6.7310e-10,  ..., 3.1865e-05, 4.8936e-04,
         2.9702e-04],
        [3.0040e-06, 5.8218e-06, 2.0156e-02,  ..., 2.1299e-17, 6.6942e-05,
         3.0086e-05],
        [2.6241e-04, 3.4477e-08, 3.2657e-11,  ..., 1.0019e-13, 1.0172e-09,
         3.0758e-08],
        ...,
        [7.9281e-07, 1.5591e-12, 9.8452e-11,  ..., 7.4900e-17, 1.4907e-06,
         2.8131e-07],
        [2.9121e-08, 1.7315e-08, 3.4201e-08,  ..., 9.1377e-01, 2.5151e-05,
         1.0455e-10],
        [5.2798e-08, 1.6000e-03, 2.1720e-05,  ..., 2.0492e-11, 3.1634e-07,
         1.2663e-06]])

In [242]:
loss = -prob[torch.arange(32), Y].log().mean()
loss

IndexError: shape mismatch: indexing tensors could not be broadcast together with shapes [32], [228146]

In [213]:
F.cross_entropy(logits, Y)

tensor(13.7715)

In [319]:
# Initialization
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27,2), generator=g)
W1 = torch.randn((X.shape[1]*C.shape[1], 100 ), generator=g)
print(W1.shape)
b1 = torch.randn(100, generator=g)

W2 = torch.randn((100,C.shape[0]), generator=g)
b2 = torch.randn(C.shape[0], generator=g)

parameters = [C, W1,b1, W2,b2]


torch.Size([6, 100])


In [320]:
for p in parameters:
    p.requires_grad = True

In [321]:
ix = torch.randint(0, X.shape[0], (32,))
ix

tensor([ 44834,   9965, 142827,  19690,  28909, 151193,  98814,  77205, 200132,
         97104, 157626, 199801,  67761, 154673,  85191, 130434, 170429, 166670,
         76583,  89408,  18961, 141856, 101277, 148829, 138831,  42289,   9829,
         52058,  16735, 200355, 152655, 118869])

In [322]:
lre = torch.linspace(-3,0, 1000)
lrr = 10**lre
len(lrr)

1000

In [326]:

#Forward Pass
lri = []
lossi = []
for i in range(10000):
    ix = torch.randint(0, X.shape[0], (32,))
    emb = C[X[ix]]
    a1 = torch.tanh(emb.view(-1,X.shape[1]*C.shape[1]) @ W1 + b1 )
    logits = a1 @ W2 + b2
    # Calculate Loss
    loss = F.cross_entropy(logits, Y[ix])
    
    # torch.zero_grad model.parameters
    
    for p in parameters:
        p.grad = None
    
    # calculate the gradients
    loss.backward()
    
    #Gradient Descent
    lr = 0.01
    for p in parameters:
        p.data += -lr*p.grad
    # lossi.append(loss.item())
    # lri.append(lr)
print(loss.item())
# plt.plot(lri, lossi)

2.1202478408813477


In [294]:
emb = C[X]
a1 = torch.tanh(emb.view(-1,X.shape[1]*C.shape[1]) @ W1 + b1 )
logits = a1 @ W2 + b2
# Calculate Loss
loss = F.cross_entropy(logits, Y)
loss.item()

3.574902057647705